In [6]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split


import pandas as pd
import os

ModuleNotFoundError: No module named 'pycaret'

In [25]:
ROOT_DIR = "./"
RANDOM_STATE = 200

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))


In [26]:
df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

normal_ratio = 1.0

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)


# df_train, df_val = train_test_split(
#     df_concat,
#     test_size=0.3,
#     stratify=df_concat["target"],
#     random_state=RANDOM_STATE,
# )

In [51]:
df_concat.tail()

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
4695,Dam dispenser #2,AJX75334501,3L1XB244-1,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,19.7,15,360,1,AbNormal
4696,Dam dispenser #2,AJX75334501,3M1XC484-1,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,20.4,11,141,1,AbNormal
4697,Dam dispenser #2,AJX75334501,3I1XA581-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.1,10.0,204,1,0,AbNormal
4698,Dam dispenser #1,AJX75334501,3G1X8303-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.4,8.0,209,1,0,AbNormal
4699,Dam dispenser #2,AJX75334501,4A1XC343-1,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,20.0,14,196,1,AbNormal


In [27]:
setup_clf = setup(data=df_concat, target="target", train_size=0.7, session_id=333, fold_shuffle=True)

,Description,Value
0,Session id,777
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(4700, 146)"
5,Transformed data shape,"(4700, 170)"
6,Transformed train set shape,"(3290, 170)"
7,Transformed test set shape,"(1410, 170)"
8,Numeric features,133
9,Categorical features,12


In [28]:
model = compare_models(sort='F1', fold=10, n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8173,0.9220,0.8173,0.8195,0.8170,0.6347,0.6368,0.0400
lightgbm,Light Gradient Boosting Machine,0.8161,0.9200,0.8161,0.8171,0.8159,0.6322,0.6333,0.2910
ridge,Ridge Classifier,0.8152,0.9229,0.8152,0.8175,0.8148,0.6304,0.6327,0.0320
rf,Random Forest Classifier,0.8112,0.9149,0.8112,0.8126,0.8110,0.6225,0.6239,0.0870
ada,Ada Boost Classifier,0.8109,0.9180,0.8109,0.8112,0.8109,0.6219,0.6222,0.1020
gbc,Gradient Boosting Classifier,0.8103,0.9193,0.8103,0.8112,0.8102,0.6207,0.6215,0.3280
et,Extra Trees Classifier,0.8073,0.8988,0.8073,0.8083,0.8071,0.6146,0.6156,0.0760
dt,Decision Tree Classifier,0.8040,0.8040,0.8040,0.8045,0.8039,0.6079,0.6084,0.0390
lr,Logistic Regression,0.7948,0.8879,0.7948,0.7981,0.7942,0.5897,0.5929,0.1420
qda,Quadratic Discriminant Analysis,0.7641,0.9160,0.7641,0.8371,0.7505,0.5283,0.5967,0.0440


In [29]:
tuned_model = [tune_model(i) for i in model]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6201,0.6681,0.6201,0.6207,0.6197,0.2403,0.2408
1,0.6049,0.6543,0.6049,0.6057,0.6042,0.2099,0.2107
2,0.6201,0.6739,0.6201,0.6209,0.6195,0.2403,0.2411
3,0.6201,0.6553,0.6201,0.6202,0.6199,0.2400,0.2402
4,0.5836,0.6378,0.5836,0.5836,0.5835,0.1671,0.1671
5,0.6383,0.6816,0.6383,0.6383,0.6383,0.2766,0.2766
6,0.6353,0.6657,0.6353,0.6411,0.6312,0.2701,0.2760
7,0.6444,0.6746,0.6444,0.6444,0.6444,0.2888,0.2888
8,0.6170,0.6901,0.6170,0.6172,0.6169,0.2341,0.2343


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8267,0.9235,0.8267,0.8268,0.8267,0.6535,0.6535
1,0.8024,0.9022,0.8024,0.8075,0.8017,0.6050,0.6100
2,0.8116,0.9190,0.8116,0.8116,0.8115,0.6231,0.6232
3,0.8389,0.9323,0.8389,0.8395,0.8388,0.6779,0.6785
4,0.8085,0.9257,0.8085,0.8111,0.8081,0.6171,0.6197
5,0.8298,0.9364,0.8298,0.8302,0.8297,0.6595,0.6600
6,0.7781,0.8967,0.7781,0.7793,0.7778,0.5561,0.5574
7,0.8602,0.9350,0.8602,0.8610,0.8601,0.7203,0.7212
8,0.8207,0.9214,0.8207,0.8212,0.8206,0.6413,0.6419


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8389,0.9361,0.8389,0.8418,0.8386,0.6779,0.6807
1,0.7964,0.9061,0.7964,0.8023,0.7954,0.5929,0.5988
2,0.8207,0.9206,0.8207,0.8207,0.8207,0.6413,0.6414
3,0.8389,0.9346,0.8389,0.8405,0.8387,0.6779,0.6794
4,0.7872,0.9246,0.7872,0.7916,0.7865,0.5746,0.5789
5,0.8207,0.9254,0.8207,0.8221,0.8204,0.6413,0.6427
6,0.7781,0.9070,0.7781,0.7818,0.7773,0.5561,0.5598
7,0.8116,0.9252,0.8116,0.8153,0.8110,0.6230,0.6267
8,0.8298,0.9259,0.8298,0.8329,0.8294,0.6595,0.6626


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8389,0.9281,0.8389,0.8395,0.8388,0.6779,0.6785
1,0.7872,0.9023,0.7872,0.7883,0.7871,0.5745,0.5756
2,0.8085,0.9168,0.8085,0.8086,0.8085,0.6170,0.6171
3,0.8237,0.9271,0.8237,0.8237,0.8237,0.6474,0.6474
4,0.8146,0.9234,0.8146,0.8149,0.8146,0.6292,0.6295
5,0.8267,0.9304,0.8267,0.8268,0.8267,0.6535,0.6535
6,0.7933,0.8978,0.7933,0.7933,0.7933,0.5866,0.5867
7,0.8389,0.9288,0.8389,0.8395,0.8388,0.6778,0.6784
8,0.8267,0.9251,0.8267,0.8277,0.8266,0.6534,0.6544


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8267,0.8985,0.8267,0.8303,0.8263,0.6536,0.6571
1,0.7872,0.8788,0.7872,0.7972,0.7855,0.5747,0.5845
2,0.8298,0.9097,0.8298,0.8412,0.8284,0.6598,0.6710
3,0.8116,0.9048,0.8116,0.8273,0.8093,0.6233,0.6388
4,0.8146,0.9044,0.8146,0.8296,0.8125,0.6294,0.6441
5,0.8298,0.8791,0.8298,0.8337,0.8292,0.6595,0.6634
6,0.7781,0.8687,0.7781,0.7846,0.7768,0.5560,0.5626
7,0.8116,0.8979,0.8116,0.8138,0.8112,0.6230,0.6253
8,0.8359,0.9128,0.8359,0.8376,0.8356,0.6717,0.6735


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [39]:
a = [tuned_model[1], tuned_model[3]]

In [40]:
blended_soft = blend_models(estimator_list=a,
                       fold=10,
                       method="hard",
                       optimize="F1")

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:54:18
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8359,0.0000,0.8359,0.8391,0.8355,0.6718,0.6750
1,0.7933,0.0000,0.7933,0.7998,0.7923,0.5868,0.5931
2,0.8207,0.0000,0.8207,0.8227,0.8204,0.6414,0.6434
3,0.8298,0.0000,0.8298,0.8316,0.8296,0.6596,0.6614
4,0.8146,0.0000,0.8146,0.8199,0.8139,0.6293,0.6345
5,0.8359,0.0000,0.8359,0.8383,0.8355,0.6716,0.6741
6,0.7933,0.0000,0.7933,0.7968,0.7927,0.5865,0.5900
7,0.8480,0.0000,0.8480,0.8522,0.8475,0.6959,0.7002
8,0.8176,0.0000,0.8176,0.8214,0.8171,0.6351,0.6390


In [41]:
final_model = finalize_model(blended_soft)
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [42]:
# Prepare training data
# features = []

# for col in df_train.columns:
#     try:
#         df_train[col] = df_train[col].astype(int)
#         features.append(col)
#     except:
#         continue

test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

In [43]:
test_data.loc[:,'Chamber Temp. Judge Value_AutoClave'].nunique()

2

In [44]:
((test_data=='OK').sum()>0).sum()

1

In [45]:
train_columns = train_data.columns
test_data = test_data[train_columns]
#test_data.loc[:,'Chamber Temp. Judge Value_AutoClave'] = test_data.loc[:,'Chamber Temp. Judge Value_AutoClave'].astype(float)

In [46]:
df_test_x = test_data.drop(columns=['target'])

In [47]:
#df_test_x.to_csv('tmp.csv', index=False)

In [48]:
test_pred = predict_model(final_model, data=df_test_x)


[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9


In [49]:
test_pred

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,prediction_label
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,90,10,...,91.800003,270.000000,50,85,19.799999,13.000000,195,1,0,AbNormal
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,14,256,1,Normal
2,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,-90,10,...,91.800003,270.000000,50,85,19.700001,1.000000,98,1,0,AbNormal
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,90,10,...,50.000000,91.800003,270,50,85.000000,20.000000,14,0,1,AbNormal
4,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,-90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,1,215,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,90,10,...,50.000000,91.800003,270,50,85.000000,19.500000,14,131,1,AbNormal
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,90,16,...,50.000000,91.800003,270,50,85.000000,19.799999,12,279,1,Normal
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,-90,16,...,50.000000,91.800003,270,50,85.000000,20.500000,4,66,1,AbNormal
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,-90,10,...,91.800003,270.000000,50,85,18.900000,1.000000,117,1,0,AbNormal


In [50]:
df_sub = pd.read_csv("../data/submission.csv")
df_sub["target"] = test_pred["prediction_label"]
df_sub.to_csv("../data/automl/submission-4.csv", index=False)